In [2]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"]="2"
import cv2
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt
from cuml.cluster import KMeans
import shutil
import glob
from cucim.skimage.transform import warp
one_seg_only=True
one_seg_rem=False
seg_img=False
from cucim.skimage import registration
from skimage import io
img_list=[]
size='3328'
orient='Horizontal'
layers=['DAPI','CD3']
import tkinter
from tkinter import filedialog
tkinter.Tk().withdraw()
#path1=os.path.join("D:\\","Users","Abu Bakr","Desktop","Raw images")
path1=filedialog.askdirectory(title='path for raw images')
img_list=[x for x in sorted(glob.glob(path1+"/*.png"))]

path2=filedialog.askdirectory(title='path for processed images')



In [3]:
from math import sqrt




def closest_color(centers):
    print(centers)
    closest_color_ctrs=[]
    i=0
    for rgb in centers:
        r, g, b = rgb
        color_diffs_pctr = []
        
            #compensate for float
        color_diff = sqrt(abs(r)**2 + abs(g)**2 + abs(b)**2)
        
        closest_color_ctrs.append((color_diff,i))
        i=i+1
    print(max(closest_color_ctrs)[0])
        #if(min(closest_color_ctrs)[0]<200):
    return max(closest_color_ctrs)[1]
      #  else:
         #   return -1

In [4]:
img_store=0
for img_no in img_list:
    print(img_no)
    if(os.path.exists(os.path.join(path2,f"{img_store:06d}"))==False):
        os.makedirs(os.path.join(path2,f"{img_store:06d}"),exist_ok=False)
    img=cv2.imread(img_no)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    pixel_values = img.reshape((-1, 3))
    pixel_values = cp.float32(pixel_values)
    
    
    
    k=4
    #print("input:")
    #print(b)

    print("Calling fit")
    kmeans_float = KMeans(n_clusters=k,max_iter=30,tol=0.000001)
    kmeans_float.fit(pixel_values)

    labels=kmeans_float.labels_
    centers=kmeans_float.cluster_centers_
    
    
    
    #centers = np.uint8(centers)
    blue_idx=closest_color(centers)
    
    
    # flatten the labels array --->Check
    labels = labels.flatten()

    segmented_image = centers[labels.flatten()]
    segmented_image = segmented_image.reshape(img.shape)
    cv2.imwrite(os.path.join(path2,f"{img_store:06d}","segmented_image "+f"{img_store:06d}"+".png"),segmented_image)
    if(one_seg_only==True):
        for i in range (k):
            if(seg_img==True):
                temp = cp.copy(segmented_image) # segmented_image flattens the image, ie processes the image and brings down the number of 
                                    # colors to k colors and changes each cluster's color to the centroid
            else:
                temp=cp.copy(img) #for real image
        # convert to the shape of a vector of pixel values
            temp = temp.reshape((-1, 3))
        # color (i.e cluster) to disable
            cluster = i
            #temp[labels == cluster] = [0, 0, 0] #for removing color belonging to a cluster

            temp[labels == cluster] = [0, 0, 0] # for removing any color not belonging to cluster


    # convert back to original shape
            temp = temp.reshape(img.shape)
            t2=cv2.cvtColor(temp,cv2.COLOR_RGB2BGR)
               # t2=cv2.medianBlur(t2,25)
            if(i==blue_idx):
                cv2.imwrite(os.path.join(path2,f"{img_store:06d}","blue_nuclei"+".png"),t2)
            else:
                cv2.imwrite(os.path.join(path2,f"{img_store:06d}","seg_one_only "+f"{i:03d}"+".png"),t2)
    if(one_seg_rem==True):
        for i in range (k):
            if(seg_img==True):
                temp = cp.copy(segmented_image) # segmented_image flattens the image, ie processes the image and brings down the number of 
                                    # colors to k colors and changes each cluster's color to the centroid
            else:
                temp=cp.copy(img) #for real image
        # convert to the shape of a vector of pixel values
            temp = temp.reshape((-1, 3))
        # color (i.e cluster) to disable
            cluster = i
            temp[labels == cluster] = [0, 0, 0] #for removing color belonging to a cluster

            #temp[labels != cluster] = [0, 0, 0] # for removing any color not belonging to cluster


    # convert back to original shape
            temp = temp.reshape(img.shape)
            t2=cv2.cvtColor(temp,cv2.COLOR_RGB2BGR)
               # t2=cv2.medianBlur(t2,25)
            cv2.imwrite(os.path.join(path2,f"{img_store:06d}","seg_one_rem "+f"{i:03d}"+".png"),t2)
    cv2.imwrite(os.path.join(path2,f"{img_store:06d}","he"+".png"),img)
    img_store=img_store+1

/home/saab/Desktop/Processed_Data/Preprocessed_Data/Validation/HE/001.png
Calling fit
[[201.22824  154.96536  215.05205 ]
 [ 61.792023   6.289359 151.15836 ]
 [230.64894  225.72853  234.33215 ]
 [163.18256   79.4377   197.40895 ]]
398.8281105157704
/home/saab/Desktop/Processed_Data/Preprocessed_Data/Validation/HE/002.png
Calling fit
[[228.81404   221.36606   233.17538  ]
 [161.82011    74.24599   198.37508  ]
 [ 64.45794     5.7999306 157.2547   ]
 [196.41296   148.48468   215.5418   ]]
394.62584451318907
/home/saab/Desktop/Processed_Data/Preprocessed_Data/Validation/HE/003.png
Calling fit
[[134.53412  111.993095 186.55292 ]
 [217.29173  191.96654  229.32683 ]
 [183.9822   153.091    212.8552  ]
 [237.60754  233.71185  240.56073 ]]
411.0328940534253
/home/saab/Desktop/Processed_Data/Preprocessed_Data/Validation/HE/004.png
Calling fit
[[221.97931 197.92374 229.88428]
 [201.8731  166.62758 219.7149 ]
 [237.6357  234.44461 240.29694]
 [146.12755 116.77742 191.84473]]
411.31206779531635
/h

img_list_blue=[os.path.join(path2,f"{img_store:03d}","blue_nuclei"+".png" ) for img_store in range(len(img_list))]


img_list_dapi=[x for x in sorted(glob.glob(os.path.join(path1,"*",'dapi','*.png')) )]
img_list_cd3=[x for x in sorted(glob.glob(os.path.join(path1,"*",'cd3','*.png') ))]
#img_list_dapi=[x for x in sorted(glob.glob(os.path.join(path1,'dapi','tiles','*.png')) )]
#img_list_cd3=[x for x in sorted(glob.glob(os.path.join(path1,'cd','tiles','*.png') ))]
#img_list_cd45=[x for x in sorted(glob.glob(os.path.join(path1,'cd45','*.png')) )]

img_list_dapi

img_store=0
for img1,img2 in zip(img_list_blue,img_list_dapi):
    path3=os.path.join(path2,f"{img_store:03d}",'align')
    if(os.path.exists(os.path.join(path3))==False):
        os.makedirs(os.path.join(path3),exist_ok=False)
    img1=cv2.imread(img1,cv2.IMREAD_GRAYSCALE)
    img2=cv2.imread(img2,cv2.IMREAD_GRAYSCALE)
    img1=cv2.resize(img1,(3072,3072))
    img2=cv2.resize(img2,(3072,3072))
    rh1,img1_1 = cv2.threshold(img1,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    rh2,img2_1 = cv2.threshold(img2,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    #img1_1=cv2.bitwise_not(img1_1)
    #img2_1=cv2.bitwise_not(img2_1)
    cv2.imwrite(os.path.join(path3,"img1 of"+f"{img_store:03d}"+".png"),img1_1)
    cv2.imwrite(os.path.join(path3,"img2 of"+f"{img_store:03d}"+".png"),img2_1)
    
    print("thresholding done for "+f"{img_store:03d}")
# --- Compute the optical flow
    #v, u = registration.optical_flow_tvl1(img1_1, img2_1,attachment=5, tightness=0.3, num_warp=5, num_iter=10, tol=0.0001)
    v, u = registration.optical_flow_tvl1(cp.asarray(img1_1), cp.asarray(img2_1))
    print("alignment done for "+f"{img_store:03d}")
# --- Use the estimated optical flow for registration

    nr, nc = img1.shape
    row_coords, col_coords = cp.meshgrid(cp.arange(nr), cp.arange(nc),indexing='ij')
    image1_warp = warp(cp.asarray(img2_1), cp.array([row_coords + v, col_coords + u]),mode='nearest')
    img1_rgb=cv2.cvtColor(img1_1,cv2.COLOR_GRAY2BGR)
    image1_warp=image1_warp.astype(np.float32)
    img2_rgb=cv2.cvtColor(cp.asnumpy(image1_warp),cv2.COLOR_GRAY2BGR)
    red = np.zeros((3072,3072,3),dtype=cp.float32)
    red[:]=np.asarray((255,0,0))
    blue = np.zeros((3072,3072,3),dtype=cp.float32)
    blue[:]=np.asarray((0,0,255))
    plt.imsave(os.path.join(path3,"overlap of"+f"{img_store:03d}"+".png"),cv2.bitwise_not(img1_rgb)*blue+
               cv2.bitwise_not(img2_rgb)*red)
    plt.imsave(os.path.join(path3,"warp of"+f"{img_store:03d}"+".png"),img2_rgb,cmap='gray')
    img_to_process=[]
    img_to_process.append(img_list_dapi[img_store])
    img_to_process.append(img_list_cd3[img_store])
    #img_to_process.append(img_list_cd45[img_store])
    processed_imgs=[]
    for img2_main in img_to_process:
        img2_main=cv2.imread(img2_main)
        img2_main=cv2.resize(img2_main,(3072,3072))
        img2_r,img2_g,img2_b=cv2.split(img2_main)
        
        image1_r_warp = warp(cp.asarray(img2_r), cp.array([row_coords + v, col_coords + u]),
                       mode='nearest')
        plt.imsave(os.path.join(path3,"image_r of"+f"{img_store:03d}"+".png"),cp.asnumpy(image1_r_warp),cmap='gray')
        image1_g_warp = warp(cp.asarray(img2_g), cp.array([row_coords + v, col_coords + u]),
                       mode='nearest')
        plt.imsave(os.path.join(path3,"image_g of"+f"{img_store:03d}"+".png"),cp.asnumpy(image1_g_warp),cmap='gray')
        image1_b_warp = warp(cp.asarray(img2_b), cp.array([row_coords + v, col_coords + u]),
                       mode='nearest')
        plt.imsave(os.path.join(path3,"image_b of"+f"{img_store:03d}"+".png"),cp.asnumpy(image1_b_warp),cmap='gray')
        img1=cv2.imread(os.path.join(path3,"image_r of"+f"{img_store:03d}"+".png"))
        img2=cv2.imread(os.path.join(path3,"image_g of"+f"{img_store:03d}"+".png"))
        img3=cv2.imread(os.path.join(path3,"image_b of"+f"{img_store:03d}"+".png"))
        img1=cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
        img2=cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
        img3=cv2.cvtColor(img3,cv2.COLOR_BGR2GRAY)
        np.multiply(img1, 1, out=img1, casting="unsafe")
        np.multiply(img2, 1, out=img2, casting="unsafe")
        np.multiply(img3, 1, out=img3, casting="unsafe")
        after = cv2.merge([img1,img2,img3])
        processed_imgs.append(after)
    for img_present,toi in zip(processed_imgs,layers):
                cv2.imwrite(os.path.join(path3,toi+f"{img_store:03d}"+".png"),img_present)
    img_store=img_store+1